<a href="https://colab.research.google.com/github/laxmi1707/RAG_ApplicationForPDF/blob/main/Apple_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is to do a RAG application over some apple report . It provides domain specific review.
Problem statement
Develop a RAG application to help business analysts efficiently extract key insights from extensive reports, such as How Apple is Organized for Innovation, enabling faster and more informed decision-making.

In [2]:
#For GPU llma cpu python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.85 --force-reinstall --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 149.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 143.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 196.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 170.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 205.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machin

In [3]:
!pip install huggingface_hub==0.23.2 pandas==1.5.3 tiktoken==0.6.0 pymupdf==1.25.1 langchain==0.1.1 langchain-community==0.0.13 chromadb==0.4.22 sentence-transformers==2.3.1 numpy==1.25.2 q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 4.3 MB/s eta 0:00:00
     ━

In [1]:
import json,os
import numpy as np
import tiktoken
import pandas as pd


#To do chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
#This is to load pdf
from langchain_community.document_loaders import PyMuPDFLoader
#For the embeddings
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
#For the vector DB
from langchain_community.vectorstores import Chroma

#download the model from hf
from huggingface_hub import hf_hub_download
#this model is from hf
from llama_cpp import  Llama


#Data Prep for RAG

## Loading the data

In [2]:
from google.colab import drive

apple_pdf_path = "HBR_How_Apple_Is_Organized_For_Innovation-4.pdf"
path ="/content/drive/MyDrive/AIMLUseCaseProjects/DataSetRAG/"
drive.mount('/content/drive')


pdf_loader = PyMuPDFLoader(path+apple_pdf_path)
apple= pdf_loader.load()

Mounted at /content/drive


#Data Overview

##Checking the first 3 pages

In [3]:
for i in range(3):
  print(f"Page num:{i+1}", end="\n")
  print(apple[i].page_content, end="\n")

Page num:1
REPRINT R2006F
PUBLISHED IN HBR
NOVEMBER–DECEMBER 2020
ARTICLE
ORGANIZATIONAL CULTURE
How Apple Is 
Organized  
for Innovation
It’s about experts leading experts. 
by Joel M. Podolny and Morten T. Hansen
This article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.

Page num:2
2
Harvard Business Review
November–December 2020
This article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.

Page num:3
PHOTOGRAPHER MIKAEL JANSSON
How Apple Is  
Organized  
for Innovation
It’s about experts  
leading experts.
ORGANIZATIONAL 
CULTURE
Joel M. 
Podolny
Dean, Apple 
University
Morten T. 
Hansen
Faculty, Apple 
University
AUTHORS
FOR ARTICLE REPRINTS CALL 800-988-0886 OR 617-783-7500, OR VISIT HBR.ORG
Harvard Business Review
November–December 2020  3
This article is made available to you with compliments of Apple

In [4]:
#See page 6
print(f"Page 6 ",apple[7].page_content)

Page 6  40 specialist teams: silicon design, camera software, reliabil-
ity engineering, motion sensor hardware, video engineering, 
core motion, and camera sensor design, to name just a few. 
How on earth does Apple develop and ship products that 
require such coordination? The answer is collaborative 
debate. Because no function is responsible for a product or a 
service on its own, cross-functional collaboration is crucial.
When debates reach an impasse, as some inevitably do, 
higher-level managers weigh in as tiebreakers, including at 
times the CEO and the senior VPs. To do this at speed with 
sufficient attention to detail is challenging for even the best 
of leaders, making it all the more important that the company 
fill many senior positions from within the ranks of its VPs, 
who have experience in Apple’s way of operating.
However, given Apple’s size and scope, even the executive 
team can resolve only a limited number of stalemates. The 
many horizontal dependencies mean th

In [5]:
#No of pages
len(apple)

11

##Chunking the data

In [6]:
#Data chunking

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name ="cl100k_base",
    chunk_size=512,
    chunk_overlap=20
)

In [7]:
document_chunks= pdf_loader.load_and_split(text_splitter)

In [8]:
len(document_chunks)

25

In [9]:
document_chunks[0].page_content

'REPRINT R2006F\nPUBLISHED IN HBR\nNOVEMBER–DECEMBER 2020\nARTICLE\nORGANIZATIONAL CULTURE\nHow Apple Is \nOrganized  \nfor Innovation\nIt’s about experts leading experts. \nby Joel M. Podolny and Morten T. Hansen\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.'

In [10]:
document_chunks[1].page_content

'2\nHarvard Business Review\nNovember–December 2020\nThis article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted.'

In [11]:
document_chunks[-1].page_content

'to cultivate the experts-leading-experts model even within \na business unit structure. For example, when filling the next \nsenior management role, pick someone with deep expertise \nin that area as opposed to someone who might make the best \ngeneral manager. But a full-fledged transformation requires \nthat leaders also transition to a functional organization. \nApple’s track rec\xadord proves that the rewards may justify the \nrisks. Its approach can produce extraordinary results.\u2002\nHBR Reprint R2006F\nJOEL M. PODOLNY is a vice president of Apple and the dean  \nof Apple University. Prior to joining Apple, in 2009, he was  \nthe dean of the Yale School of Management and on the faculty of \nHarvard’s and Stanford’s business schools. MORTEN T. HANSEN  \nis a member of Apple University’s faculty and a professor at the \nUniversity of California, Berkeley. He was formerly on the faculties  \nof Harvard Business School and INSEAD.\nFOR ARTICLE REPRINTS CALL 800-988-0886 OR 617-783

##As expected there are some overlaps

This article is made available to you with compliments of Apple Inc for your personal use. Further posting, copying or distribution is not permitted

#If we increase the chunk length then overlap increases

## Embeddings

In [12]:
embedding_model = SentenceTransformerEmbeddings(model_name="thenlper/gte-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)

In [14]:
print("Dimensions of the embedding vector", len(embedding_1))
print("Dimensions of the embedding vector", len(embedding_2))

print("Embedding vector",embedding_1)
print("Embedding vector",embedding_2)

Dimensions of the embedding vector 1024
Dimensions of the embedding vector 1024
Embedding vector [-0.007790964096784592, -0.0288176741451025, -0.027877110987901688, -0.02246643416583538, 0.022144237533211708, -0.006249097175896168, 0.008647195994853973, 0.016467047855257988, 0.018209118396043777, 0.053493041545152664, 0.013901167549192905, -0.00016959481581579894, 0.007971166633069515, -0.028715034946799278, 0.01758621260523796, -0.00955386646091938, -0.011853267438709736, -0.010937211103737354, 0.010688173584640026, 0.006078250240534544, 0.01815146952867508, 0.008558928966522217, -0.059489812701940536, -0.023468146100640297, -0.05630359798669815, 0.06647870689630508, 0.026126118376851082, 0.010339600034058094, 0.09469927102327347, 0.060104191303253174, -0.030409671366214752, -0.04133659973740578, 0.018987996503710747, -0.0794404149055481, -0.017287464812397957, -0.05677452310919762, 0.04051440581679344, -0.03622545301914215, -0.04527204483747482, -0.028730036690831184, -0.000266915187

Dimensions of embedding vector is 1024
All vector dimensions are same



*   The embedding vector provides a fixed length vector


## **Vector Database**

In [15]:
# Uninstall existing chromadb to avoid conflicts
!pip uninstall chromadb -y
# Install a specific, known-working version of chromadb
!pip install chromadb==0.4.18 -q

Found existing installation: chromadb 0.4.22
Uninstalling chromadb-0.4.22:
  Successfully uninstalled chromadb-0.4.22
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 27.0 MB/s eta 0:00:00


In [16]:
!pip show chromadb

Name: chromadb
Version: 0.4.18
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: bcrypt, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, overrides, posthog, pulsar-client, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [17]:
out_dir = 'apple_db'

In [18]:
if not os.path.exists(out_dir):
  os.makedirs(out_dir)


In [19]:
vectorstore = Chroma.from_documents(document_chunks,
                                    embedding_model,
                                    persist_directory=out_dir)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [20]:
vectorstore = Chroma(persist_directory=out_dir,embedding_function=embedding_model)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [21]:
vectorstore.embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Normalize()
), model_name='thenlper/gte-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False)

In [22]:
vectorstore.similarity_search("Apple Steve Jobs iPhone ",k=3)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(page_content='WHY A FUNCTIONAL ORGANIZATION?\nApple’s main purpose is to create products that enrich \npeople’s daily lives. That involves not only developing \nentirely new product categories such as the iPhone and the \nApple Watch, but also continually innovating within those \ncategories. Perhaps no product feature better reflects Apple’s \ncommitment to continuous innovation than the iPhone cam-\nera. When the iPhone was introduced, in 2007, Steve Jobs \ndevoted only six seconds to its camera in the annual keynote \nevent for unveiling new products. Since then iPhone camera \ntechnology has contributed to the photography industry \nwith a stream of innovations: High dynamic range imaging \n(2010), panorama photos (2012), True Tone flash (2013), opti-\ncal image stabilization (2015), the dual-lens camera (2016), \nportrait mode (2016), portrait lighting (2017), and night mode \n(2019) are but a few of the improvements.\nTo create such innovations, Apple relies on a struct

From the retrieved chunk we noticed as k=3 , we have been prvided with 3 output content chunks with metadata.
All the chunks are related to query words we provided i.e apple , steve jobs , iphone

##Retriever

In [23]:
retirever = vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k':1}
)

In [24]:
rel_docs = retirever.get_relevant_documents("How does does Apple develop and ship products that requires good coordination between the teams? ")
rel_docs

[Document(page_content='40 specialist teams: silicon design, camera software, reliabil-\nity engineering, motion sensor hardware, video engineering, \ncore motion, and camera sensor design, to name just a few. \nHow on earth does Apple develop and ship products that \nrequire such coordination? The answer is collaborative \ndebate. Because no function is responsible for a product or a \nservice on its own, cross-functional collaboration is crucial.\nWhen debates reach an impasse, as some inevitably do, \nhigher-level managers weigh in as tiebreakers, including at \ntimes the CEO and the senior VPs. To do this at speed with \nsufficient attention to detail is challenging for even the best \nof leaders, making it all the more important that the company \nfill many senior positions from within the ranks of its VPs, \nwho have experience in Apple’s way of operating.\nHowever, given Apple’s size and scope, even the executive \nteam can resolve only a limited number of stalemates. The \nmany

K is the hyperparameter we need to get best context
We observe if k=2 it will provide 2 relevant chunks of document which is best fit to answer the query

##Defining the Response Generator

Downloading and Loading the model

In [29]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"


In [30]:
model_path = hf_hub_download(repo_id=model_name_or_path,filename=model_basename)

mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

In [31]:
llm = Llama(
    model_path = model_path,
    n_ctx=2300,
    n_gpu_layers=38,
    n_batch=512
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [32]:
llm("How does apple develop and ship products?").choices[0]['text']

{'id': 'cmpl-3146915e-37a7-4236-9f53-530ecf421f5e',
 'object': 'text_completion',
 'created': 1751164330,
 'model': '/root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q6_K.gguf',
 'choices': [{'text': "\n\nApple Inc. is a technology company known for designing, manufacturing, and marketing consumer electronics, computer software, and online services. The company's approach to product development and shipping can be broken down into several stages:\n\n1. Research and Development (R&D): Apple invests heavily in R&D to create new technologies and innovations. The company employs a large team of engineers, designers, and researchers to work on upcoming products. Apple's R&D efforts often involve partnerships with other companies and universities.\n2. Concept and Design: Once Apple has identified a potential product",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'us

check if the response belong to article or not .
After checking realized the lines are not from article
WHY ?
To resolve this lets provide more context and align the response


##System and User Prompts



Prompts guide the llm to generate more accurate responses. We do this by defining 2 parts
1. The system message describing assitant role
2. A user message template including the user question and context  

In [35]:
qna_system_mesage = """
You are helpful assitant whose work is to review the reports and provide the appropriate answers from the context.
User input might have context required by you to answer the user question.
This context will begin with token ###Context.
The context contains reference to specific portions of a document relvant to the query.

User questions will beging with ###.

Only answer from the context provided . Do not mention anything about the context in the final answer.
Also mention page number from the context meta data and name of document.
If the answer is not found , respond with "I dont know".
"""

In [36]:
qna_user_message_template = """
###Context
Here are some of the documents relevant to the question mentioned below.
{context}

###Question
{question}

"""

###Response Function

In [40]:
def generate_rag_response(user_input, k=3, max_tokens= 128, temprature=0, top_p =0.95, top_k=50):
   global qna_system_mesage, qna_user_message_template

   #retirve relevenat chunks
   relevant_doc_chunks = retirever.get_relevant_documents(query=user_input, k=k)
   context_list= [d.page_content for d in relevant_doc_chunks]

   #combine all relevant doc
   context_for_query = ". ".join(context_list)

   user_message = qna_user_message_template.replace('{context}', context_for_query)
   user_message = user_message.replace('{question}', user_input)

   prompt = qna_system_mesage + '\n' +user_message

   try:
        response = llm(
                  prompt=prompt,
                  max_tokens=max_tokens,
                  temperature=temprature,
                  top_p=top_p,
                  top_k=top_k
                  )
        #Extract and print response
        response = response['choices'][0]['text'].strip()
   except Exception as e:
        response = f'Sorry, I encountered the following error: \n {e}'

   return response

##Answer Questions using RAG



Query 1 : Who are the authors and when this article published?

In [41]:
user_input =  "Who are the authors and when this article published?"
print(generate_rag_response(user_input) )

Llama.generate: prefix-match hit


Answer:
The authors of the article are Joel M. Podolny and Morten T. Hansen. The article was published in November–December 2020. (Harvard Business Review, page 3)
